In [176]:
import importlib
import pandas as pd
import amir_tools as at
import pickle
import demo_model
import numpy as np
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import ecg_plot
import torch
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model
import argparse
import datetime

model_name = "demo_model"
# get CV splits
ecg_paths = pd.read_csv("/home/padmalab/ecg/data/interim/Alberta_ECG_afib_stemi.csv")

with open("/data/padmalab/ecg/data/processed/label_df/ECG_17_labels_25posDAD_10posED_ignore_ed_ver5.pickle", 'rb') as f:
    label_df = pickle.load(f)
    label_df["ecgid"] = label_df["ecgid"].apply(lambda x: x[2:-1])
    label_df = label_df.set_index("ecgid")
        

In [177]:
# valid_ecgs
# label_df

In [178]:
valid_ecgs = ['0e98b300-b26b-11dc-4823-010ed1ca0029',
 '4bf95100-b2fc-11dc-4823-0032a1940029',
 '6ce21380-addd-11dc-4823-005521f50029',
 '74b39200-accf-11dc-4823-011db9200029',
 'ea085f80-ac08-11dc-4823-000b598a0029',
 '183fb500-af11-11dc-4823-02d6b6110029',
 'e6a77c00-b431-11dc-4823-0020a9950029',
 'bf62da80-abf9-11dc-4823-0003a6670029',
 'c0351100-b430-11dc-4823-001920ce0029',
 'e0bb3f00-b0af-11dc-4823-021a136b0029',
 '5c8cd980-f210-11dc-4823-01fe645a0029',
 '25c1be00-f226-11dc-4823-0243e6270029',
 '8b171000-7bbd-11e1-4823-01ae2a5d0029',
 'f3a38a00-b488-11dc-4823-001969580029',
 '28e6a400-b3c5-11dc-4823-00fca1a10029',
 '0c1adb00-b6fb-11dc-4823-01364ade0029',
 '93baa280-b240-11dc-4823-009d3a490029',
 '70084580-b1c9-11dc-4823-001888f20029',
 '3e43cb80-b2fc-11dc-4823-00c42cba0029',
 'cbbb5d00-b6fa-11dc-4823-0134a4f00029',
 '8e713f80-b1cf-11dc-4823-00826c0d0029',
 '40f9f800-c88b-11dc-4823-01e87f4b0029',
 'b1588a80-c0cf-11dc-4823-01a0a30a0029',
 'd17d4680-1f15-11dd-4823-000345600029',
 '19c7f280-6551-11e0-4823-003b85d10029',
 '1a188000-9784-11e0-4823-00db00fe0029',
 '3ce2f100-9cbc-11e0-4823-00567ff80029',
 'a59d5a00-f98c-11e0-4823-000b41220029',
 '3fd68980-2ca3-11e7-4823-000427e00029',
 '26be8a00-d159-11e7-4823-000b97b30029',
 '2b96c580-ebb7-11db-4823-009fad0c0029',
 'e72e0a00-eb63-11db-4823-00df63f00029',
 '23edf880-eab3-11db-4823-005e3f6f0029',
 '57e3e900-286c-11dc-4823-002ea4be0029',
 'efedd700-e9cf-11db-4823-0006a61f0029',
 '5d78dd80-2678-11dc-4823-01d04a720029',
 '10745c00-e9e5-11db-4823-001860b00029',
 '819e0f80-286e-11dc-4823-004333010029',
 'e0993400-2676-11dc-4823-01c68d370029',
 'ca183a80-ec3a-11db-4823-01055ee00029',
 'ec83f680-2674-11dc-4823-01b9c1900029',
 '50f18680-2672-11dc-4823-01a8a9910029',
 'dd7e9800-e9e5-11db-4823-001da17b0029',
 '86dc9100-eaad-11db-4823-003887a70029',
 'c8336780-ead2-11db-4823-00b120ee0029',
 '3ae9c300-267e-11dc-4823-01efa6290029',
 'b508a380-eaa2-11db-4823-0020f97c0029',
 '1aba0400-267e-11dc-4823-01eed0630029',
 'c197a380-2670-11dc-4823-019e73220029',
 'f912d700-ed34-11db-4823-00a2bade0029',
 '8a2cf500-2678-11dc-4823-01d171ff0029',
 'e5a5b580-266d-11dc-4823-018bb4e90029',
 '025d8a00-ee82-11db-4823-003431810029',
 '92a6bd80-9308-11dd-4823-00c3e6500029',
 'e04b9380-8df9-11dd-4823-0222e1820029',
 '16084080-2a09-11de-4823-00dbe60c0029',
 '996c7500-2a8f-11de-4823-00c9df850029',
 '79d54c80-2a8f-11de-4823-00c9119b0029',
 '5b907800-ea86-11de-4823-00a679fa0029',
 '9e515880-37f3-11de-4823-001c61970029',
 'da17be80-2a8e-11de-4823-00c4fce40029',
 'f7746a80-2a45-11de-4823-0097bab10029',
 '25875d80-0536-11df-4823-02e7a85e0029',
 'e90af200-f4b7-11df-4823-00063c780029',
 '26d53d00-4c81-11df-4823-0006f5bc0029',
 '64e94e80-767d-11df-4823-00871aa20029',
 'f5a4f200-0a2e-11df-4823-006da5890029',
 '55905980-466a-11df-4823-0004399c0029',
 '90d40a80-820d-11df-4823-00039d5b0029',
 'be10bc80-429f-11df-4823-014b41590029',
 '92002780-421c-11df-4823-0366bedb0029',
 'e0056900-0512-11df-4823-00c8c5a50029',
 '5b988480-0ad6-11df-4823-00ad43460029',
 '05a1a500-0795-11df-4823-00c99dec0029',
 '4141f400-f4b8-11df-4823-00087fa80029',
 '7aa47800-82fb-11df-4823-001596d00029',
 '6486b100-8242-11df-4823-00322a260029',
 'be0be680-0512-11df-4823-00c7e6180029',
 '85b05080-09c9-11df-4823-00252b0c0029',
 'ce22b400-f656-11df-4823-004aa8e80029',
 '4f657480-aa89-11e0-4823-0325f2d30029',
 '8c17fc00-5820-11e1-4823-00563b6a0029',
 '14fca180-3baa-11e1-4823-0005d26b0029',
 '652e4680-66b7-11e1-4823-01d0909f0029',
 'ff5a4780-57f9-11e1-4823-029011b70029',
 'af0ed400-5851-11e1-4823-00e2806e0029',
 '816d2680-5c9a-11dc-4823-010ecb010029',
 'e58e3f00-7513-11dc-4823-01f94d740029',
 '56f48f00-9c33-11dc-4823-006470ee0029',
 '6d749900-a278-11dc-4823-004bbb050029',
 '4be0a980-a0e0-11dc-4823-1893ba930029',
 '478abe00-a1be-11dc-4823-004bca880029',
 '02aa5f00-bdf3-11dd-4823-03f16bf30029',
 '4e10d000-f1b8-11df-4823-015801c30029',
 'b7ba2880-2007-11e0-4823-02f17bc60029',
 'b701cd80-2192-11e0-4823-0042bbbd0029',
 '548ecb80-20c5-11e0-4823-0045a4ad0029',
 'c5d25a00-4608-11e1-4823-016e9e920029',
 '23012700-611f-11e3-4823-0338116b0029',
 'ee332100-c4be-11e3-4823-00ceb8620029',
 '83b33e80-8739-11e4-4823-0006dd950029',
 '88372280-2fba-11e5-4823-002d9c480029',
 'fef20280-2f5d-11e5-4823-009cdaed0029',
 '817a3780-dc8e-11e5-4823-0006838c0029',
 '9b90d300-a0c3-11e6-4823-0005ac810029',
 'a49a1c80-a2f3-11e7-4823-001aa10f0029',
 '0ba1fb00-f4ce-11e8-4823-000387350029',
 '0df8cf00-f97e-11e8-4823-0001b7130029',
 'ee61a680-f97d-11e8-4823-0000e8ca0029',
 '9b958580-8c4e-11e8-4823-0024a3370029',
 'b68c7880-6f88-11e9-4823-00204b840029',
 '3d157c80-e3a6-11e9-4823-0004d5f30029',
 'd9e4ca80-6f9f-11dc-4823-0011563f0029',
 '0266d600-702d-11dc-4823-001b1cde0029',
 'fb27e280-7095-11dc-4823-003bfdc00029',
 '299a0d80-3eed-11dc-4823-007fefc00029',
 '767daf80-4749-11dc-4823-00dd48360029',
 '2401c900-4805-11dc-4823-0239d1b70029',
 'f6a71380-474f-11dc-4823-0107de680029',
 '27940800-4506-11dc-4823-01d4003b0029',
 '6530a500-4741-11dc-4823-00b0713d0029',
 '4c8f0380-404c-11dc-4823-004ab4920029',
 '715f2880-4695-11dc-4823-007352070029',
 '5ef4f580-474e-11dc-4823-00fd6e4e0029',
 '80b47800-45a8-11dc-4823-00576c610029',
 '26045300-4745-11dc-4823-00c101e90029',
 '1a179180-4743-11dc-4823-00b399fc0029',
 '3b48b280-4815-11dc-4823-012a23640029',
 'ebd6b180-8f02-11dd-4823-00551b360029',
 '209e0980-8d47-11dd-4823-00d43a310029',
 '6e97b500-88e2-11dd-4823-02dfe4fd0029',
 '6f5b7180-570e-11dd-4823-005678710029',
 'bdb10c80-a8c8-11dd-4823-000762240029',
 'eb59fe80-8ea1-11dd-4823-03695b6d0029',
 '92dc4280-9a05-11dd-4823-00429f790029',
 'b9fc8380-99cc-11de-4823-01199e280029',
 '25519880-9ce0-11de-4823-035e7a410029',
 '0cc41200-e4b4-11dd-4823-00a945f70029',
 '262dfe80-9ccf-11de-4823-021602370029',
 '9a583c80-19e4-11de-4823-03ce04110029',
 'cf2b5100-2ac3-11de-4823-014820ea0029',
 'bc51bf00-19e4-11de-4823-03cee0270029',
 'f8290b00-12ca-11dc-4823-050744650029',
 '54cb3e80-93b0-11dc-4823-05bc75fb0029',
 '19f99380-11e0-11dc-4823-00af48000029',
 'd2df1780-0432-11de-4823-01bb08430029',
 'cab63b00-8b59-11e0-4823-0017aa780029',
 'f6c97500-80e9-11dc-4823-001b45f90029',
 '7c8bf700-80cf-11dc-4823-0010be2a0029',
 '6fbc4600-a1e2-11e0-4823-02ab486c0029',
 'fb19e500-a1aa-11e0-4823-01e00aa90029',
 '45b65300-2aa4-11e1-4823-01ba1dfc0029',
 '15c4e900-01b7-11e2-4823-00159e930029',
 'b2579200-2a84-11dc-4823-001feb420029',
 '663ea200-1a30-11dc-4823-00161d310029',
 '84110480-1a3a-11dc-4823-00209ad10029',
 '7ec57680-261f-11de-4823-02330c9d0029',
 'b4ffcc00-244e-11de-4823-003773090029',
 'e1c9e700-2513-11de-4823-01fb6f1d0029',
 'b80ca500-23d9-11de-4823-03670a4e0029',
 'a93e4c00-2838-11de-4823-046c8b600029',
 '87743d80-23d0-11de-4823-001fe9a20029',
 'ca008780-26fa-11de-4823-00082b840029',
 'dc06df00-87df-11e9-4823-00040cd00029',
 '47d51780-8554-11e9-4823-003dfd650029',
 '15be7e00-82e6-11e9-4823-002574600029',
 '3eac3680-8319-11e9-4823-0012c6a20029',
 '1425b080-83e1-11e9-4823-000474aa0029',
 'fa69b080-860e-11e9-4823-0045f4640029',
 'b210c800-86cc-11e9-4823-000289e60029',
 '679f0980-8488-11e9-4823-002981130029',
 '77da2e80-8864-11e9-4823-0014dadc0029',
 '829e4a80-83b0-11e9-4823-0002c1480029',
 '9d3cbc00-8216-11e9-4823-0007e5970029',
 '4dbaa980-819e-11e9-4823-000172c90029',
 'c3f31a00-e846-11db-4823-06c8da210029',
 '78b2b480-e3cb-11db-4823-00f2dffe0029',
 '59b42280-e3cb-11db-4823-00f2149e0029',
 'cf2e4300-e77d-11db-4823-06a10b960029',
 'c47c9f80-0e00-11e0-4823-00629f330029',
 'fa844880-0e2d-11e0-4823-00d2f38d0029',
 '3b767e80-dac6-11df-4823-0027396e0029',
 '40532680-daf6-11df-4823-0035d7dd0029',
 '7a760400-db9a-11df-4823-0053e16a0029',
 '938bf780-32e6-11e0-4823-009ac3c70029',
 'db3ad800-3ac2-11dc-4823-000521240029',
 'fc032d80-3ac2-11dc-4823-0005fa4b0029',
 'd1814e00-37fe-11dc-4823-002a0aed0029',
 'add52800-39f7-11dc-4823-00ef551e0029',
 '007ccd00-37c9-11dc-4823-005a8aa90029',
 'd2e20000-a80b-11dc-4823-015f00570029',
 '06afa980-a291-11dc-4823-01b2fb350029',
 '318e2f00-a444-11dc-4823-0011d68b0029',
 '51707600-1454-11dc-4823-0042a7be0029',
 'b1f3b900-143c-11dc-4823-00287da30029',
 'ef6e4a00-1501-11dc-4823-01d4f0f20029',
 'b11a7d00-161e-11dc-4823-00bfb4fe0029',
 '1da24000-a758-11dc-4823-00c31f970029',
 'ffd4db00-a757-11dc-4823-00c25c2f0029',
 'f9ea0300-a806-11dc-4823-0103ddb10029',
 '15bff980-e95d-11dc-4823-02c7fc360029',
 'a3c0f980-ac24-11dc-4823-01cf57fa0029',
 '8a8de080-b12f-11dc-4823-006e91b10029',
 '12fd2300-ace3-11dc-4823-003e089e0029',
 '37568c80-a817-11dc-4823-012777d40029',
 '1ba36d80-c4ca-11dc-4823-008ed3fa0029',
 '5205cc00-c47b-11dc-4823-0070dd460029',
 '11b73780-e70c-11db-4823-00366b7d0029',
 '3ca18b80-e70c-11db-4823-003783f10029',
 '8067b580-e74b-11db-4823-0048fc7c0029']

In [179]:
as_feats = at.load_obj('/data/padmalab/ecg/data/processed/xml_df/ecg_age_sex.pickle')
as_feats["ecgId"] = as_feats["ecgId"].apply(lambda x:x[2:-1])

label_df = label_df.loc[valid_ecgs]


In [182]:
importlib.reload(demo_model)

training_generator = demo_model.DataGenerator(valid_ecgs, label_df,n_classes=label_df.shape[1],age_sex=as_feats)
validation_generator = demo_model.DataGenerator(valid_ecgs, label_df,n_classes=label_df.shape[1],age_sex=as_feats)



In [183]:
next(iter(training_generator))[0][0].shape

(64, 4096, 12)

In [ ]:
loss = 'binary_crossentropy'

lr = 0.005
batch_size = 64
opt = Adam(lr)
callbacks = [ReduceLROnPlateau(monitor='val_loss',
                               factor=0.05,
                               patience=2,
                               min_lr=lr / 100),
             EarlyStopping(patience=5,min_delta=0.00001)]
METRICS = [
      keras.metrics.AUC(multi_label=True,),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
model = demo_model.get_model_agsx(label_df.shape[1])
model.compile(loss=loss, optimizer=opt, metrics=METRICS)

# Create log
# log_dir = "logs/fit/" +model_name+ datetime.datetime.now().strftime("_%d%H%M")

# Save the BEST model
callbacks += [ModelCheckpoint('./models/demo/%s'%model_name, save_best_only=True)]
# callbacks += [TensorBoard(log_dir=log_dir,),
#               CSVLogger('csvs/logs/training_%s.log'%model_name, append=True)]  # Change append to true if continuing training
# Save the BEST model
model.fit(x= training_generator,
                    validation_data=validation_generator,
                     epochs=70,
                    callbacks=callbacks,
                    workers=1)

# fake data generation

In [185]:
# create fake data
# import neurokit2 as nk

# num_ecgs = 100
# ecgs = []
# for i in range(num_ecgs):
#     print(i)
#     ecg = np.zeros([12,5000])
#     for j in range(12):
#         ecg[j] = nk.ecg_simulate(duration=10, sampling_rate=500, heart_rate=80)   
#     np.save("demo_data/ecg_%d"%i,ecg)


In [186]:
demo_labels = label_df[0:100]
demo_labels = demo_labels.sample(100)
demo_labels.index =  ["ecg_%d"%i for i in range(len(demo_labels))]
demo_labels.to_csv("demo_data/demo_labels.csv")

In [187]:
demo_agesex_features = as_feats[0:100]
demo_agesex_features["ecgId"] = ["ecg_%d"%i for i in range(len(demo_agesex_features))]
demo_agesex_features.to_csv("demo_data/demo_asfeats.csv",index=False)

/tmp/ipykernel_63792/1718674465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_agesex_features["ecgId"] = ["ecg_%d"%i for i in range(len(demo_agesex_features))]


# load fake data 

In [188]:
demo_agesex_features = pd.read_csv("demo_data/demo_asfeats.csv") # ecgId (used for path) + tabular features 
demo_ecgs = ["ecg_%d"%i for i in range(len(demo_agesex_features))] # paths to ecg files, assuming they're numpy arrays
demo_labels = pd.read_csv("demo_data/demo_labels.csv",index_col=0) 

In [189]:
importlib.reload(demo_model)

training_generator = demo_model.DataGenerator(demo_ecgs, demo_labels,n_classes=demo_labels.shape[1],age_sex=demo_agesex_features)
# validation_generator = demo_model.DataGenerator(demo_ecgs, label_df,n_classes=label_df.shape[1],age_sex=as_feats)

In [191]:
loss = 'binary_crossentropy'
lr = 0.005
batch_size = 64
opt = Adam(lr)
callbacks = [ReduceLROnPlateau(monitor='val_loss',
                               factor=0.05,
                               patience=2,
                               min_lr=lr / 100),
             EarlyStopping(patience=5,min_delta=0.00001)]
METRICS = [
      keras.metrics.AUC(multi_label=True,),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model = demo_model.get_model_agsx(label_df.shape[1])
model.compile(loss=loss, optimizer=opt, metrics=METRICS)

# Save the BEST model
callbacks += [ModelCheckpoint('./models/demo/%s'%model_name, save_best_only=True)]
model.fit(x= training_generator,
                    validation_data=validation_generator,
                     epochs=70,
                    callbacks=callbacks,
                    workers=1)

Epoch 1/70
1/1 [==============================] - 10s 10s/step - loss: 0.6321 - auc_1: 0.1745 - precision: 0.0880 - recall: 0.7143 - prc: 0.1187 - val_loss: 29384.8750 - val_auc_1: 0.2353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_prc: 0.0628 - lr: 0.0050
Epoch 2/70


/home/amirsalimi/miniconda3/envs/ecg3/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/amirsalimi/miniconda3/envs/ecg3/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


1/1 [==============================] - 1s 871ms/step - loss: 0.5597 - auc_1: 0.1765 - precision: 0.0000e+00 - recall: 0.0000e+00 - prc: 0.0386 - val_loss: 281545.8750 - val_auc_1: 0.2353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_prc: 0.0628 - lr: 0.0050
Epoch 3/70
1/1 [==============================] - 1s 864ms/step - loss: 0.3289 - auc_1: 0.1838 - precision: 0.0000e+00 - recall: 0.0000e+00 - prc: 0.1432 - val_loss: 442345.3438 - val_auc_1: 0.2353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_prc: 0.0628 - lr: 0.0050
Epoch 4/70
1/1 [==============================] - 1s 858ms/step - loss: 0.2183 - auc_1: 0.1919 - precision: 0.2500 - recall: 0.3810 - prc: 0.2190 - val_loss: 229676.1250 - val_auc_1: 0.2353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_prc: 0.0628 - lr: 2.5000e-04
Epoch 5/70
1/1 [==============================] - 1s 860ms/step - loss: 0.2023 - auc_1: 0.1913 - precision: 0.2632 - recall: 0.3571 - prc: 0.2347 - val_loss: 150458.1250